In [2]:
# Import packages
import ee
import geemap
import os
import json

import time
import subprocess as subp

from osgeo import gdal, gdal_array
from datetime import datetime, timedelta

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
date_tdy = datetime.today() 
date_tdy = date_tdy.strftime('%Y-%m-%d')
print(date_tdy)

ee.Initialize() # Initialize

f=  open('../../../../../src/data_preprocessing/base_geojson/TL_state_shapefile_for_clip.geojson')
data =json.load(f)

data = data['features'][0]['geometry']['coordinates'][0]
roi = ee.Geometry.Polygon(data)


2023-01-12


In [5]:
collection_subset = ee.ImageCollection("MODIS/061/MOD11A1") \
    .filterDate('2012-01-01', '2022-01-01').sort('IMAGE_DATE').select('LST_Day_1km') # Only select images for the years 2016-2021
print(collection_subset.size().getInfo()) # Shows the number of images within the subcollection

3644


In [22]:
collection_subset = ee.ImageCollection("MODIS/061/MOD11A2") \
    .filterDate('2021-12-20', '2022-01-01').sort('IMAGE_DATE').select('LST_Day_1km') # Only select images for the years 2016-2021
print(collection_subset.size().getInfo()) # Shows the number of images within the subcollection

1


In [23]:
image = collection_subset.first().select('LST_Day_1km')  # Pick the first image from the 'list' and select the layer of interest
geemap.image_props(image).getInfo() # Finds basic info of this image

out = os.path.join('GEE_LSTtifs') # Set path to where we want to save the GeoTIF
# Now export each image within the collection to a GeoTIF
geemap.ee_export_image_collection(collection_subset, out_dir = out, scale=image.select('LST_Day_1km').projection().nominalScale(), region=roi, file_per_band=True, crs='EPSG:4326')

# image.select('NDVI').projection().nominalScale() sets the scale equal to the resolution of the images within the collection
# file_per_band=False: all bands are downloaded and put as one file
# file_per_band=True: each band is downloaded in a single image

Total number of images: 1

Exporting 1/1: 2021_12_27.tif
Generating URL ...
Please wait ...
Data downloaded to /home/luckyw0w/dicra/analytics/sandbox/notebooks/dppd_functions/rasters/GEE_LSTtifs




In [19]:

basepath='GEE_LSTtifs/'

if 'projected' not in os.listdir():
    os.mkdir('projected')
else:
    print('directory exists')

arr = os.listdir('GEE_LSTtifs')
for i in arr:
    cmd="gdalwarp -of GTIFF  -r cubic -t_srs '+proj=longlat +datum=WGS84 +no_defs'"+" "+basepath+str(i)+" projected/"+str(i)
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
    time.sleep(1)

# Open band 1 as array
arr = os.listdir('projected/')

if 'recalculated' not in os.listdir():
    os.mkdir('recalculated')
else:
    print('directory exists')

for i in arr:
    ds = gdal.Open('projected/'+str(i))
    b1 = ds.GetRasterBand(1)
    arr_q = b1.ReadAsArray()
   # ndv = 1
    # apply scale factpr
    data = arr_q*(0.02)  # LST Scale factor
    #data = np.where(data > 1,-9999, data)
    #data=np.where(data<0,-9999,data)
    print(str(i))
    # save array, using ds as a prototype
    output="recalculated/"+str(i)
    gdal_array.SaveArray(data.astype("float32"), output, "GTIFF", ds)

    ds = None

basepath='recalculated/'
arr = os.listdir('recalculated')

if 'clipped' not in os.listdir():
    os.mkdir('clipped')
else:
    print('directory exists')
    
for i in arr:
    cmd="gdalwarp -dstnodata -9999 -cutline tsdm/District_Boundary.shp -crop_to_cutline "+basepath+str(i)+" clipped/"+i
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
    time.sleep(1)

arr = os.listdir('clipped/')

if 'cog_lst' not in os.listdir():
    os.mkdir('cog_lst')
else:
    print('directory exists')

for i in arr:
    cmd="gdal_translate clipped/"+str(i)+" cog_lst/"+str(i)+" -co COMPRESS=LZW -co TILED=YES"
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
    time.sleep(1)

cmd="rm -r GEE_LSTtifs clipped projected recalculated"
print(cmd)
try:
    subp.check_call(str(cmd), shell=True)
except:
    print("end")

gdalwarp -of GTIFF  -r cubic -t_srs '+proj=longlat +datum=WGS84 +no_defs' GEE_LSTtifs/2021_12_31.tif projected/2021_12_31.tif
Creating output file that is 491P x 491L.
Processing GEE_LSTtifs/2021_12_31.tif [1/1] : 0Using internal nodata values (e.g. 0) for image GEE_LSTtifs/2021_12_31.tif.
Copying nodata values from source GEE_LSTtifs/2021_12_31.tif to destination projected/2021_12_31.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
2021_12_31.tif
gdalwarp -dstnodata -9999 -cutline tsdm/District_Boundary.shp -crop_to_cutline recalculated/2021_12_31.tif clipped/2021_12_31.tif
Creating output file that is 489P x 489L.
Processing recalculated/2021_12_31.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
directory exists
gdal_translate clipped/2021_12_31.tif cog_lst/2021_12_31.tif -co COMPRESS=LZW -co TILED=YES
Input file size is 489, 489
0...10...20...30...40...50...60...70...80...90...100 - done.
rm -r GEE_LSTtifs clipped projected recalculated


In [3]:
directory = 'cog_lst/'
files = os.listdir(directory)
files.sort()

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import pandas
import rasterio
from osgeo import gdal
import geopandas as gpd
from sklearn.linear_model import LinearRegression
import os
import georasters as gr
import datetime as dt
from statsmodels.tsa.seasonal import STL

/home/luckyw0w/.local/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [5]:

df = pd.DataFrame()
def data_frame(i):
    myRaster = directory+i
    elevation = gr.from_file(myRaster)
    elevation = elevation.to_pandas()
    i = i[:-4]
    #df[i] = elevation['value']
    return(elevation['value'])
#df['geometry'] = elevation['geometry']



In [6]:
import multiprocessing
import time


In [7]:
start_time = time.time()


pool = multiprocessing.Pool()

#func = partial(ndvi_dppd, boundary, tiffs, directory)

results = [result for result in pool.map(data_frame, files)]
#pool.map(func, iterable)
pool.close()
pool.join()

print("--- %s seconds ---" % (time.time() - start_time))

--- 194.012353181839 seconds ---


In [8]:
df = pd.DataFrame()
for i in range(len(results)):
    df[i] = results[i]

In [9]:
df = df.T

In [29]:
i = files[100]
myRaster = directory+i
elevation = gr.from_file(myRaster)
elevation = elevation.to_geopandas()

In [11]:
dates = [i[:4]+'-'+i[5:7]+'-'+i[8:10] for i in files]

In [12]:
df['Date'] = dates
df['Date'] = pd.to_datetime(df['Date'])

In [13]:
myRaster = directory+i
elevation = gr.from_file(myRaster)
elevation = elevation.to_pandas()

In [20]:
def Trend_score(df, DateOrdinal, i):

    sdf = df[:1510]
    res = []

    sdf = pd.Series(list(sdf[i]), index=pd.to_datetime(sdf['Date']), name="LST")
    stl = STL(sdf)
    res1 = stl.fit()
    res = res+list(res1.trend)

    sdf = df[1510:]
    sdf = pd.Series(list(sdf[i]), index=pd.to_datetime(sdf['Date']), name="LST")
    stl = STL(sdf)
    res1 = stl.fit()
    res = res+list(res1.trend)

    #Set the data in the right format for Linear Regression
    x = np.array(DateOrdinal)
    X = x.reshape(-1, 1)
    y = np.array(res)
    y = y.reshape(-1, 1)

    #Perform Linear Regression and obtain the slope
    reg = LinearRegression().fit(X, y)
    y_pred_trend = reg.predict(X)
    slope, intercept = np.polyfit(x, y_pred_trend,1)
    return slope[0]

In [15]:
DateOrdinal = [dt.datetime.toordinal(i) for i in df['Date']]

In [24]:
slopes = [Trend_score(df, DateOrdinal, i) for i in df.columns[:-1]]

In [26]:
res = np.array(slopes)
data_norm = np.where(res >= 0, res/np.max(res), -res/np.min(res))
data_norm = data_norm * -1

In [30]:
elevation
deviance = gpd.GeoDataFrame()
deviance['deviance'] = data_norm
deviance['geometry'] = elevation['geometry']

In [8]:
deviance.to_file('lst_1km_deviance.json')

In [2]:
deviance = gpd.read_file('lst_1km_deviance.json')

In [6]:
deviance['deviance'] = np.array(deviance['deviance'])* -1